## 1. Connect to SQL

In [59]:
%pip install pyodbc


Note: you may need to restart the kernel to use updated packages.


In [95]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=DESKTOP-ID946DB\\SQLEXPRESS01;"
    "DATABASE=FSI-PROJECT;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()
print("✅ Connected to SQL")

✅ Connected to SQL


## 2. Read CSV with ISO, Latitude, and Longitude

**Source:** [Google Public Data l Countes](https://developers.google.com/public-data/docs/canonical/countries_csv
#### In this step, we extract info from the html:
* **country name**
* **ISO code**
* **latitude**
* **longitude**

#### Load it into a DataFrame using **`pandas.read_html`**.


In [97]:
import pandas as pd

url = "https://developers.google.com/public-data/docs/canonical/countries_csv"
tables = pd.read_html(url) 

df_geo = tables[0]  # Use the first table

df_geo.head(1)

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra


## 3. Clean and rename columns 
#### Normalization for merge compatibility with SQL data


In [8]:
df_geo_clean = df_geo.rename(columns={
    'name': 'country', 
    'country': 'iso'   # (ISO-2)
})

# lowercase and strip
df_geo_clean['country'] = df_geo_clean['country'].str.strip().str.lower()

df_geo_clean.head(3)

,iso,latitude,longitude,country
0,AD,42.546245,1.601554,andorra
1,AE,23.424076,53.847818,united arab emirates
2,AF,33.939110,67.709953,afghanistan


## 4. Query countries from SQL and normalize names to do the merge

In [57]:
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy*")

# Country names from SQL to a DataFrame 
query = "SELECT country FROM dbo.FSI_NORMALIZED_PY"
df_sql_countries = pd.read_sql(query, conn)

# lowercase and strip
df_sql_countries['country'] = df_sql_countries['country'].str.strip().str.lower()

print(df_sql_countries.head())


                     country
0                      sudan
1  congo democratic republic
2              cote d'ivoire
3                       iraq
4                   zimbabwe


## 5. Join the data using normalized country name

In [60]:
# Merge SQL with url data
df_merged = pd.merge(
    df_sql_countries,
    df_geo_clean,
    on='country', # merge using country as reference
    how='inner'  # Keep only matching countries
)

print(df_merged.head())
print(f"Total matched countries: {len(df_merged)}")



    country iso   latitude  longitude
0     sudan  SD  12.862807  30.217636
1      iraq  IQ  33.223191  43.679291
2  zimbabwe  ZW -19.015438  29.154857
3      chad  TD  15.454166  18.732207
4   somalia  SO   5.152149  46.199616
Total matched countries: 2961


## 5.1. Verify Uniqueness
##### Using ISO, We'll Check if an ISO code is linked to more than one country name, and print the differences.


In [65]:
conflicts = {}  # store ISO codes with >1 country name

for iso_code, group in df_merged.groupby('iso'):  # iterate ISO
    unique_countries = group['country'].dropna().unique()  # get unique country names
    if len(unique_countries) > 1:
        conflicts[iso_code] = unique_countries  # store ISO with multiple country names

if conflicts:
    print("ISO codes with conflicting country names:")
    for iso, countries in conflicts.items():
        print(f"ISO: {iso} → Countries: {list(countries)}")  # show ISO and the countries
else:
    print(" ► All ISO codes are consistently linked to a single country name.")  # everything is oki doki


 ► All ISO codes are consistently linked to a single country name.


## 5.2. Verify NaN
##### Now let's check how many rows have missing ISO codes (NaN)

In [75]:
iso_nan = df_merged['iso'].isna()  # finds the rows where the ISO code is missing
total_iso_nan = iso_nan.sum()  # counts how many rows have missing ISO codes (True values)
countries_with_nan_iso = df_merged.loc[iso_nan, 'country'].unique()  # get unique countries where ISO is NaN

print(f"► Total rows with missing ISO code (NaN): {total_iso_nan}")
print("Countries with missing ISO:")
print(countries_with_nan_iso)  # show country names with NaN in ISO

► Total rows with missing ISO code (NaN): 18
Countries with missing ISO:
['namibia']


## Note: 
#### ► Although no actual ISO duplicated were found, there were some NaN values. 
#### ► Since the NaN happens on a small sample, we'll normalize that later and we'll continue with the SQL merge.




### 6. SQL Merge

In [87]:
# Filter df_merge. Keep rows where ISO has a real value (not null nor empty)

df_valid = df_merged[df_merged['iso'].notna() & (df_merged['iso'].str.strip() != '')]


In [99]:
# Update SQL with ISO, latitude y longitude using df_valid

updated_rows = 0  

for index, row in df_valid.iterrows():
    query = """
        UPDATE dbo.FSI_NORMALIZED_PY
        SET iso = ?, latitude = ?, longitude = ?
        WHERE country = ?
    """
    cursor.execute(query, (
        str(row['iso']).strip(),
        float(row['latitude']),
        float(row['longitude']),
        str(row['country']).strip()
    ))
    updated_rows += cursor.rowcount

conn.commit()
print(f"✅ Total uptadate rows in SQL: {updated_rows}")


✅ Total uptadate rows in SQL: 52271


## 7. Verify the update in SQL

In [91]:
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy*")


query = "SELECT DISTINCT country, iso FROM dbo.FSI_NORMALIZED_PY ORDER BY country"
df_check = pd.read_sql(query, conn)

print(df_check.head(5))

# ISO assigned
iso_valid_count = df_check['iso'].notna().sum()
print(f"\n Total ISO assigned: {iso_valid_count} de {df_check.shape[0]}")


               country iso
0          Afghanistan  AF
1              Albania  AL
2              Algeria  DZ
3               Angola  AO
4  Antigua and Barbuda  AG

 Total ISO assigned: 169 de 187
